# LegPy tutorial

#### Fernando Arqueros, Jaime Rosado, Victor Moya


LegPy (Low energy gamma-ray simulation in Python) is a Monte Carlo (MC) simulation code for the transportation of gamma rays and electrons with energies up to few MeVs (depending of the medium).

Several geometries are supported (cylinder, orthohedron and sphere). Photons or electrons are produced as paralell (or divergent) beams or from isotropic sources with energies following several spectral distributions (monoenergetic, flat, exponential, etc.) including those of realistic X-ray and gamma ray sources. Particles can be transported either through a single material or in geometries filled with two different media.

The description of the MC algorithm and the validation of the various approximations are described in https://doi.org/10.1016/j.radmeas.2023.107029

This **tutorial** explains the procedure to run a simulation with the LegPy package. It will be shown that only five code lines in this notebook are sufficient. Later a number of examples will be presented as well as procedures to visualize and analyze the results of the simulation.         

## Previous steps

If you have downloaded LegPy in you PC and are runnig this notebook with Jupyter or a similar application, ignore this step. But if you are running this notebook from Google Colab, first you should clone the last version of the LegPy repository each time you inizialite Colab and add the directory where LegPy is located to the Python search path:

In [ ]:
#Only for Colab users!!
!git clone https://github.com/JaimeRosado/LegPy.git
import sys
sys.path.insert(0,'/content/LegPy')

Then, import the necessary modules to run this notebook:

In [ ]:
import LegPy as lpy
import numpy as np
import matplotlib.pyplot as plt

The user has to "construct" four objects that are the main ingredients of the MC simulation:

- The composition of the medium and their physical properties at microscopic level.
- The geometry of the medium.
- The energy spectrum of the beam crossing the medium.
- The geometry of the beam and type of particles (photons or electrons)

Let's start.

## Construction of the medium I: Composition

LegPy provides the necessary data related to the interaction of photons and electrons with a large number of atoms, compounds and mixtures from the National Institute of Standards and Technology (NIST Database) https://www.nist.gov/.

Select your media from this list

In [ ]:
lpy.List_Media()

You also may search a medium name containing a string

In [ ]:
lpy.List_Media('water')

**A) Construct the medium providing the following data:**

- `name` (as written in the above list): 'Al', 'Ge', 'Bone, Cortical (Icrp)', 'Sodium Iodide', 'C-552 Air-Equivalent Plastic', 'Water, Liquid'...
- `density` (g/cm$^3$): Optional. Default value is stored in data files. Different values can be used for testing purposes.

In [ ]:
medium = lpy.Medium(name='Aluminum Oxide')
#medium = lpy.Medium(name='Al', density=2.7)
#medium = lpy.Medium(name='Bone, Cortical (Icrp)', density=1.8)

In case of a simulation on a **two-media** geometry, both have to be constructed:

In [ ]:
medium_1 = lpy.Medium(name='Aluminum Oxide')
medium_2 = lpy.Medium(name='Al', density=2.7)

### Plot attenuation coefficients vs E

You can plot the components of the attenuation coeficients vs energy of a medium. In the next command you have to provide:

- `energies` : array of energies in MeV, better in log scale.
- `l_style` : line style for plotting ('', ':', etc.), defalut '' (solid line).
- `ph` : True or False, default to True.
- `inc` : True or False, default to True.
- `coh` : True or False, default to True. Only for NIST option.
- `pair` : True or False, default to True. Only for NIST option.
- `tot` : True or False, default to True.

Several media can be plotted in the same figure so you can define several media above (with different names) and compare attenuation coefficients.

This step is optional so you can skip it.

In [ ]:
E1 = 0.01 # MeV
E2 = 20. # MeV
energy_range = np.logspace(np.log10(E1), np.log10(E2), num=150) # 150 points in a log-scale E(MeV) between E1 and E2
medium.plot_mu(energies=energy_range)
#medium2.plot_mu(energies=energy_range, l_style=':', tot=True)

### Plot CSDA ranges vs E

You can plot the continuous slowing down approximation (CSDA) range vs energy of the selected media. In the next command you have to provide:

- `energies` : array of energies in MeV, better in log scale.
- `l_style` : line style for plotting ('', ':', etc.), defalut '' (solid line).
- `units` : 'cm' or 'gcm2'. Default is 'cm'.

Several media can be plotted in the same figure so you can define several media above (with different names) and compare ranges (in the same units).

This step is optional so you can skip it.

In [ ]:
E1 = 0.01 # MeV
E2 = 20. # MeV
energy_range = np.logspace(np.log10(E1), np.log10(E2), num=150) # 150 points in a log-scale E(MeV) between E1 and E2
medium.plot_R(energies=energy_range)
#medium2.plot_R(energies=energy_range, l_style = ':')

## Construction of the medium II: Geometry

Several geometries are available: cylinder, orthohedron and sphere.

**Note**: *This geometry can be filled either with **one** single **medium** or contained **two media** splitted by an interface surface. The location of this surface has to be given for the construction of the geometry object*

**B) Select your geometry**

### Cylinder





Cylinder oriented with its axis along the z axis and its base at z=0. You have to provide:

- `r` or `diam` (cm): radius or diameter.
- `z` (cm): height.

In the **two-media** case the interface can be a horizontal surface at a given depth:

- `z_int` (cm): depth of the interface, that is, medium_1 in [0, z_int] and medium_2 in [z_int, z].

or a cylinder with a given radius:

- `r_int` (cm): medium_1 in the radial interval [0, r_int] and medium_2 in [r_int, r].

For this geometry, you may choose either cylindrical or cartesian voxelization for the energy deposit matrix. Cylindrical voxelization is appropriate for parallel beams along the z axis and isotropic sources located at the z axis. In this case, you have to input the number of intervals along the coordinates r and z:

- `n_r`.
- `n_z`.

Cartesian voxelization can also be applied in any situation and medium geometry. Here, you have to provide:

- `n_x`.
- `n_y`.
- `n_z`.

Choose your option and construct the geometry.

In [ ]:
geometry = lpy.Geometry(name='cylinder', z=2., r=1., n_z=50, n_r=1) # Cylindrical voxelization
#geometry = lpy.Geometry(name='cylinder', z=3., r=2., r_int = 0.5, n_x=10, n_y=10, n_z=10) # Cartesian voxelization

### Orthohedron

Rectangular parallelepiped oriented with its longitudinal axes parallel to the x, y, z axes. The center of bottom side is assumed to be at the origin of coordinates (0,0,0). In this geometry, only the cartesian voxelization is supported. You have to provide the dimensions of the orthohedron and the number of intervals along each axis:

- `x` (cm).
- `y` (cm).
- `z` (cm).

In the **two-media** case the interface is a horizontal surface at a given depth:

- `z_int` (cm): depth of the interface, that is, medium_1 in [0, z_int] and medium_2 in [z_int, z].

Cartesian voxelization is used in this geometry. Here, you have to provide:
- `n_x`.
- `n_y`.
- `n_z`.

In [ ]:
geometry = lpy.Geometry(name='orthohedron', x=10., y=10., z=10., z_int = 3., n_x=10, n_y=10, n_z=10)

### Sphere





Sphere centered at the origin of coordinates (0,0,0). Both cartesian and spherical voxelization can be chosen. So you have to provide either:

- `r` or `diam` (cm).
- `n_r`.

or:

- `r` (cm).
- `n_x`.
- `n_y`.
- `n_z`.

In the **two-media** case the interface can be a flat surface at a given depth:

- `z_int` (cm): z location of the interface, that is, medium_1 in the z interval [-r, z_int] and medium_2 in [z_int, r].

or a spherical surface with a given radius:

- `r_int` (cm): medium_1 in the sphere [0, r_int] and medium_2 in [r_int, r].

In [ ]:
geometry = lpy.Geometry(name='sphere', r=15.72, n_r=15) # Spherical voxelization
#geometry = lpy.Geometry(name='sphere', diam=10., r_int = 3., n_x=10, n_y=10, n_z=10) # Cartesian voxelization

#### Plot the geometry


Plot the geometry in the reference coordinate system. This step is optional.

In [ ]:
geometry.plot();

## Construction of the beam I:  Energy spectrum

**C) Select the energy spectrum of the particle beam**

from one of the following options:



### Monoenergetic



Input parameters:
- `E` (MeV).

In [ ]:
spectrum = lpy.Spectrum(name = 'mono', E = 1.)

### Multi-monoenergetic




Input parameters:
- `E_w`: energies (MeV) and their corresponding weights in a numpy array.

In [ ]:
E_w = np.array([[0.511, .80], [1.25, 0.20]]) # [[E1, w1], [E2, w2],....]
spectrum = lpy.Spectrum(name = 'multi_mono', E_w = E_w)



### Flat





Input parameters:
- `E_min` (MeV).
- `E_max` (MeV).

In [ ]:
spectrum = lpy.Spectrum(name = 'flat', E_min = 0.1, E_max = 1.0)



### Gaussian profile.





Input parameters:
- `E_mean` (MeV).
- `E_sigma` (MeV).

Internal cut: 2 x E_mean > E > 0.

In [ ]:
spectrum = lpy.Spectrum(name = 'gaussian', E_mean = 0.5, E_sigma = 0.03)



### Exponential





$I(E) \propto  e^{-E/E_{ch}}$, with E_min < E < E_max.

Input parameters:

- `E_min` (MeV).
- `E_max` (MeV).
- `E_ch` (MeV).

In [ ]:
spectrum = lpy.Spectrum(name = 'exponential', E_min = 0.1, E_max = 1.0, E_ch = 0.5)



### Reciprocal




$ I(E) \propto \frac{1}{E} $, with E_min < E < E_max.

Input parameters:
- `E_min` (MeV).
- `E_max` (MeV).

In [ ]:
spectrum = lpy.Spectrum(name = 'reciprocal', E_min = 0.01, E_max = 15.)



### From a file






A number of realistic spectra are provided in LegPy

- X-Ray spectra generated with SpekPy Web 2.08: For example 80kV_3.0Al is for a tube potential of 80kV on Tungsten (default) with a filter of 3.0 mm Aluminum. In 60kV-Mo_0.04Mo both target and filter are Molybdenum.
- Gamma-ray spectra of clinical accelerators: For example 10MV_VARIAN is a photon spectrum of a VARIAN accelerator of 10MV, 6MeV_VARIAN is an electron spectrum of 6MeV. More details of these spectra are given in the header of the corresponding .txt file inside the folder LegPy/beam_spectra.

Check the available spectra:

In [ ]:
lpy.List_Spectra()

and select one of them:

In [ ]:
spectrum = lpy.Spectrum(name = 'from_file', file = '6MV_Elekta')

If you want to use your own spectrum file, you should pass its path to the parameter `file`. The file must be a txt file with two columns: Energy (MeV)   ------   Relative Intensity (au)

In [ ]:
#This only works if there is a spectrum file called my_spectrum.txt in the working directory
#spectrum = lpy.Spectrum(name = 'from_file', file = 'test')

#### Plot the spectrum







You can plot the energy spectrum of incident beam. Again, just to check it is OK.

A number of photons are generated randomly following the requested spectrum in logaritmic scale. You should input:
- `n_part` : number of particles, default to 10^5.
- `n_bin` : number of intervals, default to 50.

In [ ]:
spectrum.plot(n_part = 100000, n_bin = 50)

## Construction of the beam II:  Geometry

**D) Select the particle (photon or electron) and the geometrical properties of the beam.**

Input parameter in all beam geometries:
- `particle`= 'photon' or 'electron'. Default is `particle` = 'photon'.

NOTE: In order not to waste computing time the beam geometry has to be defined in such a way that all particles reach the medium.

### Parallel beam

Parallel beam with entrance plane perpendicular to z axis. In general not applicable for the sphere.

Input parameters:

- `theta` (degrees) : zenith angle from z axis, default to 0.
- `phi` (degrees) : azimuth angle from x axis, default to 0.
- `p_in` (cm) : numpy array with the coordinates of the center of the beam cross section at the entrance plan, default to (0,0,0).
- `diam` (cm) : beam diameter, default to 0. (i.e., pencil beam).

In [ ]:
beam = lpy.Beam(name = 'parallel')
#beam = lpy.Beam(particle='electron', name = 'parallel')
#beam = lpy.Beam(name = 'parallel', theta = 15.0, phi = 30.0, p_in = np.array([0.1, -0.1, 0.0]))

###  Divergent beam / Isotropic source

Three options are available:

1) **Divergent beam** with the focus located on the z(<0) axis. Not applicable for the sphere.

Input parameters:

- `p_in` (cm): numpy array with the coordinates of the focus location below the XY plane (z<0). Default to x=y=0 (source on the z axis). Small (x,y) values are also possible but might increase the computing time.      
- size of the field located on the XY plane, centered at (0,0):
    - if circular, provide `diam` (cm)
    - if rectangular, provide `x_ap`, `y_ap` (cm) size

In [ ]:
z = 25. # cm
diam = 1. # cm
beam = lpy.Beam(name = 'isotropic', diam = diam, p_in = np.array ([0., 0., -z]))
#beam = lpy.Beam(name = 'isotropic',  particle='electron', x_ap = 1., y_ap = 0.5, p_in = np.array ([1., 0., -z]))

2) Isotropic **source** located **outside the medium**. Not applicable for the sphere. Currently only available for point sources. 

It is geometrically equals to the **divergent beam**

Input parameters:
- `p_in` (cm): numpy array with the coordinates of the source location below the XY plane (z<0). Default to x=y=0 (source on the z axis). Small (x,y) values are also possible but might increase the computing time.
- size of the entrance aperture located on the XY plane, centered at (0,0):
    - if circular, provide `diam` (cm)
    - if rectangular, provide `x_ap`, `y_ap` (cm) size

In [ ]:
z = .5 # cm
x, y = 0.05, -0.01 # cm
beam = lpy.Beam(name = 'isotropic', diam = 1., p_in = np.array ([x, y, -z]))
#beam = lpy.Beam(name = 'isotropic', particle='electron', x_ap = 1., y_ap = 0.5, p_in = np.array ([0., 0., -z]))

3) Isotropic **source** located **inside the medium**.

Both point sources or uniformly distributed in a sphere, cylinder or orthohedron are allowed. 

Input parameters:

- `p_in` (cm): numpy array with the coordinates of the location of a point source (or the center of a finite size  source). Default to x = y = z = 0 (source on the coordinates origin).
- size of the source. Default is point source
    - if orthohedron, provide `x_s`, `y_s`, `z_s` (cm) size.
    - if cylinder, provide `r_s`, `z_s` (cm) size.
    - if sphere, provide `r_s`, (cm) radius.

In [ ]:
x, y, z = 0.3, -0.3, 0.5 # cm
beam = lpy.Beam(name = 'isotropic', particle='photon', p_in = np.array ([x, y, z]), r_s = 5.)
#beam = lpy.Beam(name = 'isotropic')
#beam = lpy.Beam(name = 'isotropic', particle='electron', p_in = np.array ([0., 0., 0.]), z_s = 1., r_s = 5.)

#### Plot a few tracks


Check a few (50) beam tracks into the medium with the geometry you have just constructed. Optional

In [ ]:
lpy.Plot_beam(medium, geometry, spectrum, beam)

## Monte Carlo Simulation


It transports the particle beam through the medium (or media) defined above.

Parameters to be provided:

For both particle beams, **photons and electrons**:  

- `n_part`: number of particles (simulation cases). Default is `n_part` = 100.
- `E_cut`: Energy cut in MeV. Default is `E_cut` = 0.01 MeV.
- `tracks`: Plot tracks of the particle beam (not advised for > 100 photons). Default is `tracks` = False.

In case of a **photon** beam:

- `n_ang`: Number of intervals to construct angular histograms of photons leaving the medium. Default is 20.
- `n_E`: Number of energy intervals to construct several histograms. It applies to histogram of photons leaving the medium, spectra of absorbed energy and spectral fluence. Default is 20.
- `n_zloc` : Number of locations along the z axis to evaluate the photon fluence. Default is 20. Note that `n_zloc` is different from `n_z` defined in the `Geometry` object.
- `points` : Plot interaction points (not advised for > 100 photons). Default is `points` = False.
- `gamma_out` : If True, a plot angle vs energy for outgoing photons is made. Default is `gamma_out` = False. Do not use with > 10000 beam photons.
- `x_ray`: If False, X-rays from atomic de-excitation after photoelectric absorption are neglected and the whole photon energy is absorbed in the interaction point. Default is `x_ray` = True. Only K$\alpha$ and K$\beta$ photons are generated.
- `e_transport`: If True, all secondary electrons (photoelectric absorption and Compton scattering) are transported. By default (`e_transport` = False) electron energy is absorbed at the interaction point.

In case of an **electron** beam:

- `n_ang`: Number of intervals to construct histograms of angular distribution of emerging electrons. Default is 20.
- `n_zloc`: Number of z positions/depths to construct transmission plots. Default is 20.
- `e_length`, `e_K` : The electron transport is performed by steps of either same length (`e_length`) or same energy loss fraction (`e_K`). By default, e_length is obtained from the voxel size, but these parameters may also be specified, if desired. The parameter `e_length` is in $\mu$m.

Next some examples:

Simulation on **one** single **medium**:

In [ ]:
result = lpy.MC(medium, geometry, spectrum, beam, n_part=1000, n_E=50)
#result = lpy.MC(medium, geometry, spectrum, beam, n_part=10000, e_transport = True)

In case of a **two-media** geometry:

In [ ]:
result = lpy.MC([medium_1, medium_2], geometry, spectrum, beam, n_part=1000)
#result = lpy.MC([medium_1, medium_2], geometry, spectrum, beam, n_part=int(1.e5), n_zloc = 100)

Return:

An object (dubbed **result**) containing

If **photon** beam:

- the spatial distribution of deposited energy
- the angular and energy spectra of outgoing photons
- the histogram of absorbed energy
- the spectral fluence along the z axis.

If **electron** beam:

- the spatial distribution of deposited energy
- the histogram of electron ranges
- the angular histogram of backscattered electrons.

Once this object is generated, this information can be plotted or stored in files (see examples below).

# Examples

Once the user gets familiar with the construction of the above objects and the MC parameters, running a simulation can be easily performed. Next we show a few **examples**:  

- If not yet you should import several modules

In [ ]:
import LegPy as lpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Only for Colab users!!
!pip install xlsxwriter
import xlsxwriter

## 1) Photon beam - one medium

### 1.1) Display of photons tracks and/or interaction points

In [ ]:
medium = lpy.Medium(name='Sodium Iodide')
geometry = lpy.Geometry(name='cylinder', diam=50., z=20., n_x=30, n_y=30, n_z=100)
beam = lpy.Beam(name='parallel', theta=0.0, phi=0.0, diam=4., p_in=(0.,0.,0.))
spectrum = lpy.Spectrum(name='exponential', E_min=0.1, E_max=2., E_ch=0.3)
lpy.Plot_beam(medium, geometry, spectrum, beam, n_part=100, points=True) #tracks=True by default
#Alternative:
#result = lpy.MC(medium, geometry, spectrum, beam, n_part=100, tracks=True, points=False)

### 1.2) Plot of angle vs energy of outgoing photons

In [ ]:
medium = lpy.Medium(name='Bismuth Germanium Oxide')
geometry = lpy.Geometry(name='cylinder', diam=50., z=1., n_x=30, n_y=30, n_z=100)
beam = lpy.Beam(name='parallel', theta=0.0, phi=0.0)
spectrum = lpy.Spectrum(name='mono', E=1.)
result = lpy.MC(medium, geometry, spectrum, beam, n_part = 1000, gamma_out=True)

### 1.3) Histograms

In [ ]:
medium = lpy.Medium(name='Water, Liquid')
geometry = lpy.Geometry(name='cylinder', diam=50., z=10., n_x=30, n_y=30, n_z=100)
beam = lpy.Beam(name='parallel', theta=0.0, phi=0.0)
spectrum = lpy.Spectrum(name='mono', E=1.)
result = lpy.MC(medium, geometry, spectrum, beam, n_part=10000, n_ang=50, n_E=50)

**Histograms of angular and energy distributions of outgoing photons as well as the spectrum of energy absorption can be**

- plot

In [ ]:
result.plot_hists()

- stored in dataframes

In [ ]:
ang_out_df = result.ang_out()
ang_out_df.head()

In [ ]:
E_out_df = result.E_out()
E_out_df.head()

In [ ]:
E_ab_df = result.E_ab()
E_ab_df.head()

- written in excel files (a sheet per histogram)

In [ ]:
result.hists_to_excel("my_excel")

### 1.4) Energy deposition

#### 1.4.1) Cylindral symmetry

In [ ]:
medium = lpy.Medium(name='Tissue, Soft (Icru Four-Component)')
geometry = lpy.Geometry(name='cylinder', diam=50., z=100., n_r=30, n_z=100)
beam = lpy.Beam(name='parallel', theta=0.0, phi=0.0, diam=10.)
spectrum = lpy.Spectrum(name='exponential', E_min=0.1, E_max=2., E_ch=0.3)
result = lpy.MC(medium, geometry, spectrum, beam, n_part = 10000, e_transport=True)

#### Spatial distribution can be:

- plot

In [ ]:
result.plot_Edep()

- stored in dataframes

In [ ]:
E_dep_df = result.Edep_to_df()
E_dep_df.head()

- written in excel file

In [ ]:
result.Edep_to_excel('my_excel')

Note that above results are averaged over axial angle

#### 1.4.2) Cartesian symmetry

In [ ]:
medium = lpy.Medium(name='Tissue, Soft (Icru Four-Component)')
geometry = lpy.Geometry(name='cylinder', diam=50., z=100., n_x=30, n_y=30, n_z=100)
beam = lpy.Beam(name='parallel', theta=0.0, phi=0.0, diam=10.)
spectrum = lpy.Spectrum(name='exponential', E_min=0.1, E_max=2., E_ch=0.3)
result = lpy.MC(medium, geometry, spectrum, beam, n_part = 10000)

Spatial distribution can be:

- plot

In [ ]:
result.plot_Edep()

- stored in python binary file with extension .npy. This 3D energy distribution cannot be stored in an excel file or a dataframe, but can be saved in this file for later use.

In [ ]:
result.Edep_to_npy('my_3D_Edep')

Total energy deposited in the medium can be calculated

In [ ]:
Vp = geometry.delta_v # pixel volume (cm^3)
Ed = result.Edep.sum() * Vp
print('total energy deposit =', round(Ed, 3), 'keV')

Personalized plots of the spatial distribution of Edep can be done with this tool:  

In [ ]:
z_ind = [0, 4, 9, 13, 16] # index of z layers.
prof_lev = [3.5, 4., 5., 6., 7.] # dosis levels (adjust according the color bar)
result.plot_Edep_layers(axis="z", indexes=z_ind, c_profiles=True, lev=prof_lev)

In [ ]:
x_ind = [2, 9, 10, 15, 20] # index of z layers.
prof_lev = [3.5, 4., 5., 6., 7.] # dosis levels (adjust according the color bar)
result.plot_Edep_layers(axis="x", indexes=x_ind, c_profiles=True, lev=prof_lev)

### 1.5) Fluence

In [ ]:
medium = lpy.Medium(name='Water, Liquid')
geometry = lpy.Geometry(name='cylinder', diam=50., z=50., n_r=30, n_z=50)
beam = lpy.Beam(name='parallel', theta=0.0, phi=0.0)
spectrum = lpy.Spectrum(name='exponential', E_min=0.1, E_max=2., E_ch=0.3)
result = lpy.MC(medium, geometry, spectrum, beam, n_part = 10000, n_zloc=10)

Total and spectral photon fluence is evaluated along the z axis in n_zloc+1 locations (from z = 0 to z = maximum depth) by counting photons traversing surfaces of the size of the cross section of the voxels, corrected for the direction of each photon. In this way it is calculated for all these z location the number of photons per unit area perpendicular to the surface.

- Fluence plot

    - First histogram: total fluence as function of depth
    - Second histogram: spectral fluence at the entrance (z=0)
    - Third histogram: spectral fluence at the exit (maximum z)

In [ ]:
result.plot_fluence()

- Fluence data can be stored in a dataframe or in an excel file

In [ ]:
fluence_df = result.fluence_to_df()
fluence_df.head()

In [ ]:
result.fluence_to_excel("my_excel")

## 2) Electron beam - one medium

### 2.1) Display of electron tracks

In [ ]:
medium = lpy.Medium(name='Al')
geometry = lpy.Geometry(name='orthohedron', x =.2, y = 0.2, z=.2, n_x=30, n_y=30, n_z=100)
spectrum = lpy.Spectrum(name='mono', E = 1.)
beam = lpy.Beam(particle='electron', name='parallel', theta=30.0, phi=20.0, diam=.02, p_in=(-0.03, 0., 0.))
lpy.Plot_beam(medium, geometry, spectrum, beam, n_part=50)
#Alternative:
#result = lpy.MC(medium, geometry, spectrum, beam, n_part=50, tracks=True)

### 2.2) Electron range and backscattering studies ###

First, check the CSDA range for an appropriate choice of the size of the medium and the `e_length` (`e_K`) parameters   

In [ ]:
E = 1.0 #MeV
spectrum = lpy.Spectrum(name='mono', E=E)
medium = lpy.Medium(name='Al') # NIST
e_data = medium.e_data
CSDA = np.interp(E, e_data.E_ref, e_data.R_ref) # cm
CSDAum = CSDA * 1.e4 # um
print('CSDA = ', round(CSDA, 3), 'cm')

A step length of CSDA/100 (or `e_K` = 0.95) should be enough to get accurate results

In [ ]:
e_length = CSDAum * 0.01 # um (1/100 of CSDA)
#e_K = 0.95

In [ ]:
geometry = lpy.Geometry(name='cylinder', diam=.25, z=.25, n_x=20, n_y=20, n_z=20)
beam = lpy.Beam(particle = 'electron', name='parallel')
n_part = 3000
result = lpy.MC(medium, geometry, spectrum, beam, n_part=n_part, n_zloc=50, e_length=e_length)

- plot histograms of range, transmission and backscattered angle

In [ ]:
result.plot_hists()

#### 2.2.1) Electron range ####

Depending on the practical case, two different definitions of electron range R are used:

- The depth z of the electron at the end of its path. This definition is used for the first histogram of the above figure.
- The maximum depth reached by the electron. This definition is associated to the transmission curve obtained experimentally when the number of electrons traversing layers of several depths z are measured and is used for the second histogram of the above figure.

Differences between both ranges might be non-negligible in cases with strong backscattering.  

Both definitions of range can be computed and stored in a dataframe with three components: x = R(cm), y = number of electrons, z = fraction of electrons.

Check its shape

In [ ]:
range_df = result.final_z()
#range_df = result.max_z()
range_df.head()

Plot of R distribution and the corresponding integral function of the electron fraction vs R in either definition. Similar to to the plots obtained above but you can personalized them.  

In [ ]:
range_df.plot(kind='scatter', x=0, y=1);
range_df.plot(kind='scatter', x=0, y=2);

Calculation of the **extrapolated range** and other parameters

In [ ]:
ext_R, mode, av = lpy.ext_range(range_df)

This funcion is also incorporated in the result object. The range `definition` should be set to "final" (default) or "max".

In [ ]:
ext_R, mode, av = result.ext_range(definition="max")

#### 2.2.2) Electron Backscattering ####

The angle distribution of backscattered electrons can be computed and stored in a dataframe.

In [ ]:
back = result.backscattering()
back.head()

The backscattering coefficiente, b, is the fraction of backscattered electrons.

In [ ]:
b = back.sum()[1]/n_part
print('b = ', round(100.*b, 2), '%')

Plot the angular distribution of backscattered electrons

In [ ]:
back.plot(kind='scatter', x=0, y=2);

### 2.3) Energy deposition ###

#### 2.3.1) Cylindrical symmetry   ####

In [ ]:
medium = lpy.Medium(name='Al')
geometry = lpy.Geometry(name='cylinder', r =.2, z=.2, n_r=20, n_z=20)
spectrum = lpy.Spectrum(name='mono', E=1.)
beam = lpy.Beam(particle='electron', name='parallel', diam=0.02)
result = lpy.MC(medium, geometry, spectrum, beam, n_part=10000)

Spatial distribution can be:

- plot

In [ ]:
result.plot_Edep()

- stored in dataframes

In [ ]:
E_dep_df = result.Edep_to_df()
E_dep_df.head()

- written in excel file

In [ ]:
result.Edep_to_excel('my_excel')

Note that above results are averaged over axial angle

#### 2.3.2) Cartesian symmetry   ####

In [ ]:
medium = lpy.Medium(name='Al')
geometry = lpy.Geometry(name='cylinder', r=0.2, z=0.2, n_x=40, n_y=40, n_z=20)
spectrum = lpy.Spectrum(name='mono', E=1.)
beam = lpy.Beam(particle='electron', name='parallel', diam=0.02)
result = lpy.MC(medium, geometry, spectrum, beam, n_part=10000)

Spatial distribution can be:

- plot

In [ ]:
result.plot_Edep()

- stored in python binary file with extension .npy. Note that this 3D energy distribution cannot be stored in an excel file or a dataframe, but can be saved in this file for later use.

In [ ]:
result.Edep_to_npy('my_3D_Edep')

Total energy deposited in the medium

In [ ]:
Vp = geometry.delta_v # pixel volume (cm^3)
Ed = result.Edep.sum() * Vp
print('total energy deposit =', round(Ed, 3), 'keV')

Personalized plots of the spatial distribution of Edep can be done with this tool:  

In [ ]:
z_ind = [0, 4, 9, 13, 16] # index of z layers.
prof_lev = [3.5, 4., 5., 6., 7.] # dosis levels (adjust according the color bar)
result.plot_Edep_layers(axis="z", indexes=z_ind, c_profiles=True, lev=prof_lev)

In [ ]:
x_ind = [4, 14, 20, 30, 35] # index of z layers.
prof_lev = [3.5, 4., 5., 6., 7.] # dosis levels (adjust according the color bar)
result.plot_Edep_layers(axis="x", indexes=x_ind, c_profiles=True, lev=prof_lev)

### 2.4) Backscattered energy ###

Similar to the above case (both cylndrical or cartesian voxelization) but forcing the electron beam to start its path in some point inside the medium     

#### 2.4.1) Cylindrical symmetry   ####

In [ ]:
medium = lpy.Medium(name='Al')
geometry = lpy.Geometry(name='cylinder', r=.2, z=.30, n_r=20, n_z=30)
spectrum = lpy.Spectrum(name='mono', E=1.)
z_in = 0.1
beam = lpy.Beam(particle='electron', name='parallel', diam=0.02, p_in=np.array([0., 0., z_in]))
result = lpy.MC(medium, geometry, spectrum, beam, n_part=10000)

In [ ]:
result.plot_Edep()

#### 2.4.2) Cartesian symmetry   ####

In [ ]:
medium = lpy.Medium(name='Al')
geometry = lpy.Geometry(name='cylinder', r=0.2, z=0.3, n_x=40, n_y=40, n_z=30)
spectrum = lpy.Spectrum(name='mono', E=1.)
z_in = 0.1
beam = lpy.Beam(particle='electron', name='parallel', diam=0.02, p_in=np.array([0., 0., z_in]))
result = lpy.MC(medium, geometry, spectrum, beam, n_part=10000)

In [ ]:
result.plot_Edep()

Personalized plots of the spatial distribution of Edep can be done with this tool:  

In [ ]:
z_ind = [4, 9, 10, 13, 25] # index of z layers.
prof_lev = [3.5, 4., 5., 6., 7.] # dosis levels (adjust according the color bar)
result.plot_Edep_layers(axis="z", indexes=z_ind, c_profiles=True, lev=prof_lev)

In [ ]:
x_ind = [4, 14, 20, 30, 35] # index of z layers.
prof_lev = [3.5, 4., 5., 6., 7.] # dosis levels (adjust according the color bar)
result.plot_Edep_layers(axis="x", indexes=x_ind, c_profiles=True, lev=prof_lev)

The energy deposited at z < z_in is the backscattered energy.

In [ ]:
Vp = geometry.delta_v # pixel volume (cm^3)
Edep = result.Edep
Ed = Edep.sum() * Vp #total Edep
Eb = Edep[:, :, 0:10].sum() * Vp #back Edep
Ef = Edep[:, :, 10:30].sum() * Vp #for Edep

print('Fraction of backsacttered energy =', round(100. * Eb / Ed, 2), '%')

## 3) Simulations on two-media geometries

The simulation on geometries with two media can be performed in a similar way but you have to:

- Define (construct) both media.
- Give the position of the interface surface in your geometry.

Note that a common voxelization in used in both media. The result can be analyze with the same tools described  above.

Next you can find some examples.

### 3.1) A divergent photon beam on a two-media cylinder ###

In [ ]:
water = lpy.Medium(name='Water, Liquid') # NIST
gold = lpy.Medium(name='Au') # NIST

In [ ]:
E = 1. # MeV
spectrum = lpy.Spectrum(name='mono', E = E)
beam = lpy.Beam(name = 'isotropic', diam = 2.0, p_in = np.array ([0., 0., -5]))

#### 3.1.1) z interface   ####

Interface: horizontal plane at z = z_int; water in [0, z_int] and gold in [z_int, z].

In [ ]:
geometry = lpy.Geometry(name='cylinder', z=2., r=2., n_z=20, n_r=20, z_int=1.5)
result = lpy.MC([water, gold], geometry, spectrum, beam, n_part=10000)
result.plot_Edep()

#### 3.1.2) r interface   ####

Interface: Inner cylinder of radius equals r_int; water in [0, r_int] and gold in [r_int, r].

In [ ]:
geometry = lpy.Geometry(name='cylinder', z=2., r=2., n_z=20, n_r=20, r_int=1.)
result = lpy.MC([water, gold], geometry, spectrum, beam, n_part=10000)
result.plot_Edep()

### 3.2) A paralell electron beam on a two-media orthohedron ###

In [ ]:
water = lpy.Medium(name='Water, Liquid') # NIST
bone = lpy.Medium(name='Bone, Compact (Icru)') # NIST

In [ ]:
E = .75 # MeV
spectrum = lpy.Spectrum(name='mono', E=E)
beam = lpy.Beam(name='parallel')

In [ ]:
geometry = lpy.Geometry(name='orthohedron', x=1., y=1., z=5., n_x=19, n_y=19, n_z=25, z_int=3.)
result = lpy.MC([water, gold], geometry, spectrum, beam, n_part=10000)
result.plot_Edep()

### 3.3) An isotropic photon-source in the center of a two media sphere. ###

In [ ]:
water = lpy.Medium(name='Water, Liquid') # NIST
gold = lpy.Medium(name='Au') # NIST

In [ ]:
E = 1. # MeV
beam = lpy.Beam(name='isotropic', p_in=np.array([0., 0., 0.]))

#### 3.3.1) r interface   ####

Interface: inner sphere of radius r_int(cm); medium_1 in the sphere [0, r_int] and medium_2 in [r_int, r].

In [ ]:
geometry = lpy.Geometry(name='sphere', r=5., n_r=50, r_int=3.)
result = lpy.MC([water, gold], geometry, spectrum, beam, n_part=10000)
result.plot_Edep()

#### 3.3.2) z interface   ####

Interface: horizontal plane at z = z_int; water in -r > z > z_int and gold in z_int > z > r.

In [ ]:
geometry = lpy.Geometry(name='sphere', diam=10., n_r=50, n_z=10, z_int=2.5)
result = lpy.MC([water, gold], geometry, spectrum, beam, n_part=10000)
result.plot_Edep()